In [1]:
import xarray as xr
import numpy as np
from xhistogram.xarray import histogram as xhist
import matplotlib.pyplot as plt
from datetime import datetime
import sys, os
sys.path.append(os.path.abspath("/work/bb1018/b380873/tropic_vis/utilities/"))
from plotting_utilities import *

### cdo commands to adjust the ICON nc variable names and contents

### Combine all ICON trajectories into 1 file

In [ ]:
# Concatenate all of the ICON-1M trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_ICON_0V1M0A0R/'
new_file = xr.open_dataset( basedir + 'traj_tst00001350_p001_trim_extract_dt.nc' )
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'traj_tst00001350_p' + traj_prefix(i) + str(i) + '_trim_extract_dt.nc' ) # ICON
    new_file = xr.concat( (new_file, file2), dim='id' )
    
display(new_file)
new_file.to_netcdf( path=basedir + 'traj_tst00001350_trim_extract_dt.nc' )

In [ ]:
# Concatenate all of the ICON-2M trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_ICON_0V2M0A0R/'
new_file = xr.open_dataset( basedir + 'traj_tst00000450_p001_trim_extract_dt.nc' )
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'traj_tst00000450_p' + traj_prefix(i) + str(i) + '_trim_extract_dt.nc' ) # ICON
    new_file = xr.concat( (new_file, file2), dim='id' )
    
display(new_file)
new_file.to_netcdf( path=basedir + 'traj_tst00000450_trim_extract_dt.nc' )

### cdo commands to adjust the CLaMS nc variables names and contents

### Combine all CLaMS trajectores into 1 file

In [ ]:
# Concatenate all of the CLAMS trajectory files into one
basedir = '/work/bb1018/b380873/traj_output/traj_CLAMS-Tf_0V2M0A0R_noSHflux/'
new_file = xr.open_dataset( basedir + 'cirrus_tst00000450_p001_trim_clams_dt_iwc.nc' ) #extract_
for i in np.arange(2,27):
    print(i)
    file2 = xr.open_dataset( basedir + 'cirrus_tst00000450_p' + traj_prefix(i) + str(i) + '_trim_clams_dt_iwc.nc' ) # extract_
    new_file = xr.concat( (new_file, file2), dim='id' )
    
#display(new_file)
new_file.to_netcdf( path=basedir + 'cirrus_tst00000450_trim_clams_dt_iwc.nc' ) # extract_

### Load all trajectories for 5 simulations from 5 files
Initiate the binning for different variables here as well.

In [3]:
# These thresholds are used throughout. Values below are considered negligible.
qi_threshold = 10**(-8)
Ni_threshold = 10**(-8)
RHi_threshold = 0

In [4]:
basedir = '/work/bb1018/b380873/traj_output/'
fichiers = [ 'CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc' ]
            #'CLAMS-Tf_0V1M0A0R_tst00001350_trim_extract_dt_iwc.nc', 'ICON_0V1M0A0R_tst00001350_trim_extract_dt.nc',
            # 'CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc.nc', 'ICON_0V2M0A0R_tst00000450_trim_extract_dt.nc',
            # 'CLAMS_0V2M0A0R_tst00000450_trim_extract_dt_iwc.nc']

# Logarithmic IWC bins in ppmv, as well as their centerpoints
qi_bins = np.logspace( -5, 3.5, 50 )
qi_bins_c = ( qi_bins[1:] + qi_bins[:-1] )/2.

# Logarithmic Ni bins in cm-3, as well as their centerpoints
Ni_bins = np.logspace( -4, 5.5, 50 )
Ni_bins_c = ( Ni_bins[1:] + Ni_bins[:-1] )/2.

# Linear T bins in K
T_bins = np.linspace( 190, 240, 50 )
T_bins_c = ( T_bins[1:] + T_bins[:-1] )/2.

# Linear RHi bins in %
RHi_bins = np.linspace( 60, 120, 50 )
RHi_bins_c = ( RHi_bins[1:] + RHi_bins[:-1] )/2.

# Flight 7 track times
time0 = datetime(2017, 8, 8, 6, 20)
timef = datetime(2017, 8, 8, 6, 48)

### Create a generically structured dataset to store histogram values and copy it 5 times

In [5]:
# Initially all the datasets contain the same variables so we can create a generic dataset...
ds_generic = xr.Dataset( 
    data_vars=dict(  qih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     Th=(['T_bin'], np.zeros(T_bins_c.shape)),
                     qih_outflow=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih_outflow=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qih_insitu=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih_insitu=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qih_flight=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nih_flight=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     RHih=(['RHi_bin'], np.zeros(RHi_bins_c.shape)) ),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c),
                  T_bin=(['T_bin'], T_bins_c),
                  RHi_bin=(['RHi_bin'], RHi_bins_c) ) )

# ... and then copy it 5 times.
datasets =  [ ds_generic ]
#[ ds_generic, ds_generic.copy(), ds_generic.copy(), ds_generic.copy(), ds_generic.copy() ]
del ds_generic

## Different 'flavors' of qi and Ni histograms calculated and mean / median qi / Ni printed

In [6]:
#RHi_1M = xr.open_dataset( basedir + fichiers[0] )['RHI']
RHi_2M = xr.open_dataset( basedir + fichiers[0] )['RHI']  # fichiers[2]
RHi_fi = [ RHi_2M ]
    #RHi_1M, RHi_1M, RHi_2M, RHi_2M, RHi_2M ]

In [7]:
%%time
# Save the qih_* values in the Datasets above
for f, d, r in zip(fichiers, datasets, RHi_fi):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg kg-1 to ppmv
    qi = fi['qi'] * 10**6
    T = fi['T']
    time = fi['time']

    # Filter for non-negligible qi (or now CLaMS RHi) values; side-note: large peak in ICON qi at 10**(-10)
    #qi = qi.where( qi > qi_threshold )
    qi = qi.where( (r > RHi_threshold ) & (qi > qi_threshold) )
    qi_outflow = qi.where( (T >= 210) & (T <= 237) )
    qi_insitu = qi.where( (T < 210) )
    qi_flight = qi.sel( time=slice(time0, timef) )
    print(np.nanmean(qi),np.nanmedian(qi))
    print(np.nanmean(qi_outflow),np.nanmedian(qi_outflow))
    print(np.nanmean(qi_insitu),np.nanmedian(qi_insitu))
    print(np.nanmean(qi_flight),np.nanmedian(qi_flight))

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_outflow, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_insitu, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_flight, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qih_flight'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, T )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( T, dim=['time','id'], block_size=100, weights=wgts, bins=[T_bins] )
    d['Th'] = h
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc
53.43395037211612 16.62167596805375
52.032821274506155 16.779730685811955
36.62998601484411 9.196852261084132
14.768917712764654 5.806602985103382
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CPU times: user 46.3 s, sys: 37.3 s, total: 1min 23s
Wall time: 1min 32s


In [8]:
%%time
# Save the Nih_* values in the Datasets above.
for f, d, r in zip(fichiers, datasets, RHi_fi):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg-1 to L-1
    rho = fi['RHO']
    Ni = fi['Ni'] * rho  / 1000.
    Ni.name = fi['Ni'].name
    T = fi['T']
    time = fi['time']
    
    # Filter for non-negligible values
    Ni = Ni.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
    Ni_outflow = Ni.where( (T >= 210) & (T <= 237) )
    Ni_insitu = Ni.where( (T < 210) )
    Ni_flight = Ni.sel( time=slice(time0, timef) )
    print(np.nanmean(Ni),np.nanmedian(Ni))
    print(np.nanmean(Ni_outflow),np.nanmedian(Ni_outflow))
    print(np.nanmean(Ni_insitu),np.nanmedian(Ni_insitu))
    print(np.nanmean(Ni_flight),np.nanmedian(Ni_flight))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_outflow, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_insitu, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_flight, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nih_flight'] = h

CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc
536.1990738865144 24.284928057732714
522.1387132623357 23.95665650941671
1575.3982748179615 61.09877820867155
242.30756702692958 11.026734980206898
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CPU times: user 37.1 s, sys: 34.4 s, total: 1min 11s
Wall time: 1min 24s


### TE and RHI histograms calculated

In [9]:
%%time
basedir = '/work/bb1018/b380873/traj_output/'
# Save the RHIh_ values for both datasets and the TEh_* values for the CLaMS Datasets only
for f, d in zip(fichiers, datasets):
    print(f)
    fi = xr.open_dataset( basedir + f )
    qi = fi['qi'] * 10**6
    RHi = fi['RHI'].where( qi > qi_threshold )
    
    wgts = 1*xr.apply_ufunc( np.isfinite, RHi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( RHi, dim=['time','id'], block_size=100, weights=wgts, bins=[RHi_bins] )
    d['RHih'] = h

for f, d in zip(fichiers, datasets):
    print(f)
    fi = xr.open_dataset( basedir + f )
    qi = fi['qi'] * 10**6
    TE = fi['TE'].where( qi > qi_threshold )
    RHi = fi['RHI'].where( qi > qi_threshold )

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, TE )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( TE, dim=['time','id'], block_size=100, weights=wgts, bins=[T_bins] )
    d['TEh'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, RHi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( RHi, dim=['time','id'], block_size=100, weights=wgts, bins=[RHi_bins] )
    d['RHih'] = h

CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc
CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc
CPU times: user 29 s, sys: 30.5 s, total: 59.5 s
Wall time: 1min 13s


### qi-Nih-T-RHi histograms saved to nc files

In [10]:
# Save the qi and Ni histogram datasets in nc files
writedir = '/work/bb1018/b380873/traj_output/traj_pp/'
names = [ 'qih-Nih-CLAMS-Tf_0V2M0A0R_noSHflux.nc' ]
    #'qih-Nih-CLAMS-Tf_0V1M0A0R2.nc', 'qih-Nih-ICON_0V1M0A0R2.nc', 
    #      'qih-Nih-CLAMS-Tf_0V2M0A0R2.nc', 'qih-Nih-ICON_0V2M0A0R2.nc',
    #      'qih-Nih-CLAMS_0V2M0A0R2.nc']
for n, d in zip(names, datasets):
    d.to_netcdf( writedir + n )

## Ice microphysical process histograms
These differ between CLaMS and ICON so we have to initially create two different structures.

In [11]:
# For CLaMS, we will look at the homogeneous, heterogeneous, and preexisiting qi and Ni
ds_clams = xr.Dataset( 
    data_vars=dict(  qhomih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nhomih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qhetih=(['qi_bin'], np.zeros(T_bins_c.shape)),
                     Nhetih=(['Ni_bin'], np.zeros(qi_bins_c.shape)),
                     qpreih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Npreih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)) ),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c) ) )

# ... and then copy it 3 times.
datasets_clams = [ ds_clams, ds_clams.copy(), ds_clams.copy() ]
del ds_clams

In [5]:
# For ICON, we will look at qi and Ni sedimentation fluxes in and out
ds_icon = xr.Dataset( 
    data_vars=dict(  qsedih=(['qi_bin'], np.zeros(qi_bins_c.shape)),
                     Nsedih=(['Ni_bin'], np.zeros(Ni_bins_c.shape)),
                     qsedoh=(['qi_bin'], np.zeros(T_bins_c.shape)),
                     Nsedoh=(['Ni_bin'], np.zeros(qi_bins_c.shape)) ),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c) ) )

# ... and then copy it 3 times.
datasets_icon = [ ds_icon, ds_icon.copy(), ds_icon.copy() ]
del ds_icon

In [ ]:
basedir = '/work/bb1018/b380873/traj_output/'
for f, d in zip(fichiers, datasets_clams):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert to ppmv / L-1 and filter for non-negligible values
    qi_hom = fi['IWC_hom'] * 10**6
    qi_hom = qi_hom.where( qi_hom > qi_threshold )
    
    qi_het = fi['IWC_het'] * 10**6
    qi_het = qi_het.where( qi_het > qi_threshold )
    
    qi_pre = fi['IWC_pre'] * 10**6
    qi_pre = qi_pre.where( qi_pre > qi_threshold )
    
    rho = fi['RHO'] # rho
    Ni_hom = fi['ICN_hom'] * rho / 1000
    Ni_hom.name = fi['ICN_hom'].name
    Ni_hom = Ni_hom.where( Ni_hom > Ni_threshold )
    
    Ni_het = fi['ICN_het'] * rho / 1000
    Ni_het.name = fi['ICN_het'].name
    Ni_het = Ni_hom.where( Ni_het > Ni_threshold )
    
    Ni_pre = fi['ICN_pre'] * rho / 1000
    Ni_pre.name = fi['ICN_pre'].name
    Ni_pre = Ni_pre.where( Ni_pre > Ni_threshold )

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_hom )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_hom, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qhomih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_het )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_het, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qhetih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_pre )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_pre, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qpreih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_hom )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_hom, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nhomih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_het )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_het, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Nhetih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_pre )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_pre, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['Npreih'] = h

CLAMS-Tf_0V2M0A0R_noSHflux_tst00000450_trim_extract_dt_iwc.nc


In [ ]:
%%time
basedir = '/work/bb1018/b380873/traj_output/'
# Save the <process>h_ values for both datasets, ICON first
for f, d in zip(fichiers[1::2], datasets_icon):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert to ppmv and filter for non-negligible values
    # We want the magnitude of these sedimentation values so multiply by -1.
    qsedi = -1*fi['qsedi'] * 10**6
    qsedi = qsedi.where( qsedi > qi_threshold )
    qsedo = -1*fi['qsedo'] * 10**6
    qsedo = qsedo.where( qsedo > qi_threshold )
    
    # As above but converting to L-1
    rho = fi['rho']
    qnsedi = fi['qnsedi'] * rho / 1000.
    qnsedi.name = fi['qnsedi'].name
    qnsedi = -1*qnsedi.where( qnsedi > Ni_threshold )               
    qnsedo = fi['qnsedo'] * rho / 1000.
    qnsedo.name = fi['qnsedo'].name
    qnsedo = -1*qnsedi.where( qnsedo > Ni_threshold )    
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qsedi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qsedi, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qsedih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qsedo )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qsedo, dim=['time','id'], block_size=100, weights=wgts, bins=[qi_bins] )
    d['qsedoh'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qnsedi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qnsedi, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['qnsedih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qnsedo )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qnsedo, dim=['time','id'], block_size=100, weights=wgts, bins=[Ni_bins] )
    d['qnsedoh'] = h

In [10]:
scratchdir = '/work/bb1018/b380873/traj_output/traj_pp/'
names = [ 'het-hom-pre-CLAMS-Tf_0V2M0A0R_noSHflux.nc' ]
    #'het-hom-pre-CLAMS-Tf_0V1M0A0R.nc', 'qsed-ICON_0V1M0A0R.nc', 
    #      'het-hom-pre-CLAMS-Tf_0V2M0A0R.nc', 'qsed-ICON_0V2M0A0R.nc',
    #      'het-hom-pre-CLAMS_0V2M0A0R.nc']
datasets = [ datasets_clams[0], datasets_icon[0], datasets_clams[1],
             datasets_icon[1], datasets_clams[2] ]
for n, d in zip(names, datasets):
    d.to_netcdf( scratchdir + n )

## Two-dimensional histograms - qi, Ni versus T, RHi

In [15]:
# Reduce the number of qi and Ni bins relative to the 1D histograms

# Logarithmic IWC bins in ppmv, as well as their centerpoints
qi_bins = np.logspace( -5, 3.5, 25 )
qi_bins_c = ( qi_bins[1:] + qi_bins[:-1] )/2.

# Logarithmic Ni bins in cm-3, as well as their centerpoints
Ni_bins = np.logspace( -4, 5.5, 25 )
Ni_bins_c = ( Ni_bins[1:] + Ni_bins[:-1] )/2.

### Using StratoClim data

In [43]:
sc = xr.open_dataset( '/work/bb1018/b380873/tropic_vis/obs/stratoclim2017.geophysika.0808_1.filtered_per_sec.nc')
sc_qi = sc['BEST:IWC']
sc_temp = sc['BEST:TEMP']

sc_qi = sc_qi.where( (sc_qi > qi_threshold) )
sc_temp = sc_temp.where( (sc_qi > qi_threshold) )

wgts = 1*xr.apply_ufunc( np.isfinite, sc_qi )
wgts = wgts / wgts.sum( dim=['time'] ) * 100.

h = xhist( sc_qi, sc_temp, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
qs = qi_bins_c.shape[0]
ts = T_bins_c.shape[0]
ds_generic = xr.Dataset( data_vars=dict( qiTh_flight=(['qi_bin','T_bin'], np.zeros((qs, ts)) ) ) )
ds_generic['qiTh_flight'] = h
ds_generic.to_netcdf( '/work/bb1018/b380873/tropic_vis/obs/stratoclim2017.geophysika.0808_1_hist.nc' )

### Using simulation trajectories

In [11]:
# Initially all the datasets contain the same variables so we can create a generic dataset...
qs = qi_bins_c.shape[0]
ts = T_bins_c.shape[0]
ns = Ni_bins_c.shape[0]
rs = RHi_bins_c.shape[0]

ds_generic = xr.Dataset( 
    data_vars=dict(  qiTh=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),
                     qiTh_outflow=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh_outflow=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih_outflow=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih_outflow=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),
                     qiTh_insitu=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh_insitu=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih_insitu=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih_insitu=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),
                     qiTh_flight=(['qi_bin','T_bin'], np.zeros(( qs, ts )) ),
                     NiTh_flight=(['Ni_bin','T_bin'], np.zeros(( ns, ts )) ),
                     qiRHih_flight=(['qi_bin','RHi_bin'], np.zeros(( qs, rs )) ),
                     NiRHih_flight=(['Ni_bin','RHi_bin'], np.zeros(( ns, rs )) ),),
    coords=dict(  qi_bin=(['qi_bin'], qi_bins_c),
                  Ni_bin=(['Ni_bin'], Ni_bins_c),
                  T_bin=(['T_bin'], T_bins_c),
                  RHi_bin=(['RHi_bin'], RHi_bins_c)) )

# ... and then copy it 5 times.
datasets = [ ds_generic, ds_generic.copy(), ds_generic.copy(), ds_generic.copy(), ds_generic.copy() ]
del ds_generic

In [ ]:
for f, d, r in zip(fichiers, datasets, RHi_fi):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg kg-1 to ppmv
    qi = fi['qi'] * 10**6
    T = fi['T']
    time = fi['time']

    # Filter for non-negligible qi (or now CLaMS RHi) values; side-note: large peak in ICON qi at 10**(-10)
    #qi = qi.where( qi > qi_threshold )
    qi = qi.where( (r > RHi_threshold ) & (qi > qi_threshold) )
    T = T.where( (r > RHi_threshold ) & (qi > qi_threshold) )
    #print('ALL')
    #print(qi.mean().values,T.mean().values)
    #print(qi.median().values,T.median().values)
    
    qi_outflow = qi.where( (T >= 210) & (T <= 237) )
    T_outflow = T.where( (T >= 210) & (T <= 237) )
    #print('OUTFLOW')
    #print(qi_outflow.mean().values,T_outflow.mean().values)
    #print(qi_outflow.median().values,T_outflow.median().values)
    
    qi_insitu = qi.where( (T < 210) )
    T_insitu = T.where( (T < 210) )
    #print('IN-SITU')
    #print(qi_insitu.mean().values,T_insitu.mean().values)
    #print(qi_insitu.median().values,T_insitu.median().values)
    
    qi_flight = qi.sel( time=slice(time0, timef) )
    T_flight = T.sel( time=slice(time0, timef) )
    #print('FLIGHT')
    #print(qi_flight.mean().values,T_flight.mean().values)
    #print(qi_flight.median().values,T_flight.median().values)

    wgts = 1*xr.apply_ufunc( np.isfinite, qi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi, T, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_outflow, T_outflow, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_insitu, T_insitu, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_flight, T_flight, block_size=100, bins=[qi_bins, T_bins], weights=wgts )
    d['qiTh_flight'] = h
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
for f, d, r in zip(fichiers, datasets, RHi_fi):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg kg-1 to ppmv
    qi = fi['qi'] * 10**6
    RHI = fi['RHI']
    T = fi['T']
    time = fi['time']

    # Filter for non-negligible qi (or now CLaMS RHi) values; side-note: large peak in ICON qi at 10**(-10)
    #qi = qi.where( qi > qi_threshold )
    qi = qi.where( (r > RHi_threshold ) & (qi > qi_threshold) )
    RHI = RHI.where( (r > RHi_threshold ) & (qi > qi_threshold) )
    
    qi_outflow = qi.where( (T >= 210) & (T <= 237) )
    RHI_outflow = RHI.where( (T >= 210) & (T <= 237) )
    
    qi_insitu = qi.where( (T < 210) )
    RHI_insitu = RHI.where( (T < 210) )
    
    qi_flight = qi.sel( time=slice(time0, timef) )
    RHI_flight = RHI.sel( time=slice(time0, timef) )

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi, RHI, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_outflow, RHI_outflow, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_insitu, RHI_insitu, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, qi_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( qi_flight, RHI_flight, block_size=100, bins=[qi_bins, RHi_bins], weights=wgts )
    d['qiRHih_flight'] = h
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
%%time
# Save the Nih_* values in the Datasets above.
for f, d, r in zip(fichiers, datasets, RHi_fi):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg-1 to L-1
    rho = fi['rho']
    Ni = fi['Ni'] * rho  / 1000.
    Ni.name = fi['Ni'].name
    T = fi['T']
    time = fi['time']
    
    # Filter for non-negligible values
    Ni = Ni.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
    T = T.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
    
    Ni_outflow = Ni.where( (T >= 210) & (T <= 237) )
    T_outflow = T.where( (T >= 210) & (T <= 237) )
    
    Ni_insitu = Ni.where( (T < 210) )
    T_insitu = T.where( (T < 210) )
    
    Ni_flight = Ni.sel( time=slice(time0, timef) )
    T_flight = T.sel( time=slice(time0, timef) )

    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni, T, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
    d['NiTh'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_outflow, T_outflow, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
    d['NiTh_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_insitu, T_insitu, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
    d['NiTh_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_flight, T_flight, block_size=100, weights=wgts, bins=[Ni_bins,T_bins] )
    d['NiTh_flight'] = h

In [ ]:
%%time
# Save the Nih_* values in the Datasets above.
for f, d, r in zip(fichiers, datasets, RHi_fi):
    print(f)
    fi = xr.open_dataset( basedir + f )
    
    # Convert kg-1 to L-1
    rho = fi['rho']
    Ni = fi['Ni'] * rho  / 1000.
    Ni.name = fi['Ni'].name
    T = fi['T']
    RHI = fi['RHI']
    time = fi['time']
    
    # Filter for non-negligible values
    Ni = Ni.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
    RHI = RHI.where( (r > RHi_threshold ) & (Ni > Ni_threshold) )
    #print('ALL')
    #print(Ni.mean().values,RHI.mean().values)
    #print(Ni.median().values,RHI.median().values)
    
    Ni_outflow = Ni.where( (T >= 210) & (T <= 237) )
    RHI_outflow = RHI.where( (T >= 210) & (T <= 237) )
    #print('OUTFLOW')
    #print(Ni_outflow.mean().values,RHI_outflow.mean().values)
    #print(Ni_outflow.median().values,RHI_outflow.median().values)
    
    Ni_insitu = Ni.where( (T < 210) )
    RHI_insitu = RHI.where( (T < 210) )
    #print('IN-SITU')
    #print(Ni_insitu.mean().values,RHI_insitu.mean().values)
    #print(Ni_insitu.median().values,RHI_insitu.median().values)
    
    Ni_flight = Ni.sel( time=slice(time0, timef) )
    RHI_flight = RHI.sel( time=slice(time0, timef) )
    #print('FLIGHT')
    #print(Ni_flight.mean().values,RHI_flight.mean().values)
    #print(Ni_flight.median().values,RHI_flight.median().values)
    #print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    
    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, qi )
    # Weight by the total number of non-nan values
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni, RHI, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
    d['NiRHih'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_outflow )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_outflow, RHI_outflow, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
    d['NiRHih_outflow'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_insitu )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_insitu, RHI_insitu, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
    d['NiRHih_insitu'] = h
    
    wgts = 1*xr.apply_ufunc( np.isfinite, Ni_flight )
    wgts = wgts / wgts.sum( dim=['time','id'] ) * 100.
    h = xhist( Ni_flight, RHI_flight, block_size=100, weights=wgts, bins=[Ni_bins,RHi_bins] )
    d['NiRHih_flight'] = h

In [60]:
scratchdir = '/scratch/b/b380873/traj_pp/'
names = [ 'qiNi-TRHi-2Dh-CLAMS-Tf_0V1M0A0R.nc', 'qiNi-TRHi-2Dh-ICON_0V1M0A0R.nc', 
          'qiNi-TRHi-2Dh-CLAMS-Tf_0V2M0A0R.nc', 'qiNi-TRHi-2Dh-ICON_0V2M0A0R.nc',
          'qiNi-TRHi-2Dh-CLAMS_0V2M0A0R.nc']
for n, d in zip(names, datasets):
    d.to_netcdf( scratchdir + n )